In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import descartes

In [2]:
# set the filepath and load in a shapefile
fp = "geo_export_fe53e91c-1d1a-470b-86d5-341e72617479.shp"
map_df = gpd.read_file(fp)
map_df.head()

,dist_label,dist_num,geometry
0,17TH,17,"POLYGON ((-87.71067 41.99737, -87.71067 41.997..."
1,20TH,20,"POLYGON ((-87.66029 41.99092, -87.66029 41.990..."
2,31ST,31,"POLYGON ((-87.82818 41.98384, -87.82816 41.983..."
3,31ST,31,"POLYGON ((-87.83365 41.97535, -87.83366 41.974..."
4,19TH,19,"POLYGON ((-87.64492 41.96973, -87.64431 41.969..."


In [3]:
# Read in the cleaned data
df_crimes = pd.read_csv("Chicago_Crime_2013.csv")
df_dispatch = pd.read_csv("cleaned_dispatch_time.csv")
df_crimes['District'].head()

0     3.0
1     3.0
2     1.0
3    17.0
4    25.0
Name: District, dtype: float64

In [4]:
# Create a new dataframe df_districts that contains number of crimes and number of arrests per district
df_districts = pd.DataFrame(columns=["dist_num", "num_crimes", 'white_per', 'dispatch_time'])
df_districts['dist_num'] = list(range(1, 26))
df_districts['num_crimes'] = 0
df_districts['num_arrests'] = 0
df_districts['white_per'] = 0
df_districts['dispatch_time'] = 0
df_districts = df_districts.set_index('dist_num')

for index, row in df_crimes.iterrows():
    district_number = int(row['District'])
    df_districts.at[district_number, 'num_crimes'] += 1
    
    if row['Arrest'] == True:
        df_districts.at[district_number, 'num_arrests'] += 1
        
for index, row in df_dispatch.iterrows():
    district_number = int(row['No.'])
    avg_dispatch_time = int((row['Jun'] + row['Jul'] + row['Aug'] + row['Sep'])/4)
    white_percentage = int(row['White %'])
    
    df_districts.at[district_number, 'white_per'] = white_percentage
    df_districts.at[district_number, 'dispatch_time'] = avg_dispatch_time
    
        
    
df_districts = df_districts.reset_index()
df_districts.head()

,dist_num,num_crimes,white_per,dispatch_time,num_arrests
0,1,4328,52,143,1198
1,2,4820,18,185,1290
2,3,5907,2,585,1540
3,4,6540,8,389,1503
4,5,4879,1,375,1355


In [ ]:
# Merge the geoframe and our dataframe
map_df['dist_num'] = map_df['dist_num'].astype('int64')
merged = map_df.merge(df_districts)

In [ ]:
variable = 'num_crimes'
# set the range for the choropleth
vmin, vmax = 1800, 8000
# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(20, 12))
merged.plot(column=variable, cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8')
ax.axis('off')
ax.set_title('Number of Chicago Crimes per District', fontdict={'fontsize': '25', 'fontweight' : '3'})
# Create colorbar as a legend
sm = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=vmin, vmax=vmax))
# empty array for the data range
sm._A = []
# add the colorbar to the figure
cbar = fig.colorbar(sm)
fig.savefig("crimes.png", dpi=300)

In [ ]:
variable = 'num_arrests'
# set the range for the choropleth
vmin, vmax = 300, 2000
# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(20, 12))
merged.plot(column=variable, cmap='Greys', linewidth=0.8, ax=ax, edgecolor='0.8')
ax.axis('off')
ax.set_title('Number of Chicago Arrests per District', fontdict={'fontsize': '25', 'fontweight' : '3'})
# Create colorbar as a legend
sm = plt.cm.ScalarMappable(cmap='Greys', norm=plt.Normalize(vmin=vmin, vmax=vmax))
# empty array for the data range
sm._A = []
# add the colorbar to the figure
cbar = fig.colorbar(sm)
fig.savefig("arrests.png", dpi=300)

In [ ]:
variable = 'white_per'
# set the range for the choropleth
vmin, vmax = 0, 100
# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(20, 12))
merged.plot(column=variable, cmap='Greens', linewidth=0.8, ax=ax, edgecolor='0.8')
ax.axis('off')
ax.set_title('White Percentage per Chicago Distract', fontdict={'fontsize': '25', 'fontweight' : '3'})
# Create colorbar as a legend
sm = plt.cm.ScalarMappable(cmap='Greens', norm=plt.Normalize(vmin=vmin, vmax=vmax))
# empty array for the data range
sm._A = []
# add the colorbar to the figure
cbar = fig.colorbar(sm)
fig.savefig("white_per.png", dpi=300)

In [ ]:
variable = 'dispatch_time'
# set the range for the choropleth
vmin, vmax = 100, 600
# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(20, 12))
merged.plot(column=variable, cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8')
ax.axis('off')
ax.set_title('Police Dispatch Time per Chicago Distract', fontdict={'fontsize': '25', 'fontweight' : '3'})
# Create colorbar as a legend
sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
# empty array for the data range
sm._A = []
# add the colorbar to the figure
cbar = fig.colorbar(sm)
fig.savefig("dispatch.png", dpi=300)

In [ ]:
df_districts.head()